# Mobility-Station-Finder

## Imports

In [1]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd
from matrixconverters.read_ptv import ReadPTVMatrix
import requests
import os

## Read and process static data

### Paths

In [4]:
path_npvm_zones = os.path.join("..", "data", "Verkehrszonen_Schweiz_NPVM_2017_shp.zip")
path_mobility_stations = os.path.join("..", "data", "mobility-stationen-und-fahrzeuge-schweiz.csv")
path_pt_jrta = os.path.join("..", "data", "140_JRTA_(OEV).mtx")
path_pt_ntr = os.path.join("..", "data", "144_NTR_(OEV).mtx")

### Read NPVM-zones with shapes

In [5]:
%time gdf_npvm_zones = gpd.read_file(path_npvm_zones, encoding="cp1252").to_crs(4326)

CPU times: total: 3.34 s
Wall time: 3.34 s


In [ ]:
gdf_npvm_zones.head()

In [ ]:
print("Anzahl NPVM-Zonen: {}".format(len(gdf_npvm_zones)))

In [ ]:
def get_npvm_zone(id_):
    return gdf_npvm_zones[gdf_npvm_zones.ID == id_]

In [ ]:
get_npvm_zone(5301003)

### Read Mobility-stations, assign NPVM-zones to Mobility-stations

In [ ]:
df_mobility_vechicles = pd.read_csv(path_mobility_stations, delimiter=";", encoding="utf8")[["Stationsnummer", "Name", "Standort"]].dropna()

In [ ]:
print("Anzahl Mobility Fahrzeuge: {}".format(len(df_mobility_vechicles)))

In [ ]:
df_mobility_stations = df_mobility_vechicles.groupby("Stationsnummer").first().reset_index()

In [ ]:
df_mobility_stations["lon"] = df_mobility_stations["Standort"].apply(lambda x: x.split(",")[1])
df_mobility_stations["lat"] = df_mobility_stations["Standort"].apply(lambda x: x.split(",")[0])
df_mobility_stations = gpd.GeoDataFrame(df_mobility_stations, geometry=gpd.points_from_xy(df_mobility_stations.lon, df_mobility_stations.lat), crs=4326)

In [ ]:
print("Anzahl Mobility Stationen: {}".format(len(df_mobility_stations)))

In [ ]:
gdf_mobilty_stations_with_npvm_zone = gpd.sjoin(df_mobility_stations, gdf_npvm_zones)[["Stationsnummer", "Name", "geometry", "ID", "N_Gem"]]

In [ ]:
print("Anzahl Mobility Standorte mit zugeordneter NPVM-Zone: {}".format(len(gdf_mobilty_stations_with_npvm_zone)))

In [ ]:
gdf_mobilty_stations_with_npvm_zone.head()

In [ ]:
gdf_npvm_zones_with_mobility_station = gdf_mobilty_stations_with_npvm_zone.dissolve(by="ID", aggfunc={"N_Gem": "first", "Name": lambda x: list(x), "Stationsnummer": lambda x: list(x)}).reset_index()

In [ ]:
print("Anzahl NPVM-Zonen mit Mobility-Standort: {}".format(len(gdf_npvm_zones_with_mobility_station)))

In [ ]:
gdf_npvm_zones_with_mobility_station[gdf_npvm_zones_with_mobility_station.N_Gem == "Samedan"]

### Read PT-skims

In [ ]:
%time skim_jrta = ReadPTVMatrix(path_pt_jrta)

In [ ]:
%time skim_ntr = ReadPTVMatrix(path_pt_ntr)

In [ ]:
def get_skim(skim_matrix, from_npvm_zone_id, to_npvm_zone_id):
    return skim_matrix.sel(origins=from_npvm_zone_id).sel(destinations=to_npvm_zone_id).matrix.item()

In [ ]:
def get_jrta(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_jrta, from_npvm_zone_id, to_npvm_zone_id)

In [ ]:
def get_ntr(from_npvm_zone_id, to_npvm_zone_id):
    return get_skim(skim_ntr, from_npvm_zone_id, to_npvm_zone_id)

In [ ]:
get_jrta(378601001, 35101026)

In [ ]:
get_ntr(378601001, 35101026)

## Execute query

### Define origin and destination and assign NPVM-Zone

In [ ]:
orig_lon_lat = (9.533490, 46.848940)
dest_lon_lat = (9.557410, 46.338920)

In [ ]:
orig_point = Point(orig_lon_lat[0], orig_lon_lat[1])
dest_point = Point(dest_lon_lat[0], dest_lon_lat[1])

In [ ]:
gdf_orig = gpd.GeoDataFrame({'geometry': [orig_point]}, crs="EPSG:4326")
gdf_dest = gpd.GeoDataFrame({'geometry': [dest_point]}, crs="EPSG:4326")

In [ ]:
gdf_orig_with_zone = gpd.sjoin(gdf_orig, gdf_npvm_zones)[["ID", "N_Gem", "geometry"]]
gdf_dest_zone = gpd.sjoin(gdf_orig, gdf_npvm_zones)[["ID", "N_Gem", "geometry"]]

In [ ]:
gdf_orig_with_zone

In [ ]:
gdf_dest_zone

In [ ]:
orig_zone_id = gdf_orig_with_zone["ID"].item()

### Compute potential NPVM-zones with Mobility-station

In [ ]:
gdf_circle_around_orig  = gpd.GeoDataFrame(geometry=gdf_orig_with_zone.to_crs(2026).buffer(5*1000).to_crs(4326), crs="EPSG:4326")
gdf_circle_around_dest = gpd.GeoDataFrame(geometry=gdf_dest_zone.to_crs(2026).buffer(100*1000).to_crs(4326), crs="EPSG:4326")

In [ ]:
gdf_circle_around_orig

In [ ]:
gdf_circle_around_dest

In [ ]:
gdf_npvm_zones_with_mobility_station_in_orig_circle = gpd.sjoin(gdf_npvm_zones_with_mobility_station, gdf_circle_around_orig)
gdf_npvm_zones_with_mobility_station_in_dest_circle = gpd.sjoin(gdf_npvm_zones_with_mobility_station, gdf_circle_around_dest)

In [ ]:
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Startpunkts: {}".format(len(gdf_npvm_zones_with_mobility_station_in_orig_circle)))
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Zielpunkts: {}".format(len(gdf_npvm_zones_with_mobility_station_in_dest_circle)))

In [ ]:
gdf_npvm_zones_with_mobility_station_in_orig_circle.head()

In [ ]:
gdf_npvm_zones_with_mobility_station_in_dest_circle[gdf_npvm_zones_with_mobility_station_in_dest_circle.N_Gem == "Samedan"]

In [ ]:
gdf_npvm_zones_with_potential_mobility_station = pd.concat([gdf_npvm_zones_with_mobility_station_in_orig_circle, gdf_npvm_zones_with_mobility_station_in_dest_circle]).groupby("ID").first().reset_index().drop(["index_right"], axis=1)

In [ ]:
print("Anzahl NPVM-Zonen mit Mobility-Station im Umkreis des Start- oder Zielpunkts: {}".format(len(gdf_npvm_zones_with_potential_mobility_station)))

In [ ]:
gdf_npvm_zones_with_potential_mobility_station.head()

In [ ]:
gdf_npvm_zones_with_potential_mobility_station[gdf_npvm_zones_with_potential_mobility_station.ID == 30601007]

### Get Mobility travel time and distance from ORMS

In [ ]:
list_npvm_zones_with_potential_mobility_station = list(gdf_npvm_zones_with_potential_mobility_station.to_records())

In [ ]:
list_npvm_zones_with_potential_mobility_station[:3]

In [ ]:
coords_str = "{},{}".format(dest_point.x, dest_point.y)
for pot_mob_st in list_npvm_zones_with_potential_mobility_station:
    center = pot_mob_st[2].centroid
    coords_str += ";{},{}".format(center.x, center.y)
url = "https://router.project-osrm.org/table/v1/driving/{}?destinations=0&annotations=duration,distance".format(coords_str)

In [ ]:
%time res = requests.get(url).json()

In [ ]:
road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id = {x[1]: res["distances"][x[0] + 1][0] for x in list_npvm_zones_with_potential_mobility_station}
road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id = {x[1]: res["durations"][x[0] + 1][0] for x in list_npvm_zones_with_potential_mobility_station}

In [ ]:
print("Anzahl Distanzen: {}".format(len(road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id)))
print("Anzahl Reisezeiten: {}".format(len(road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id)))

In [ ]:
gdf_npvm_zones_with_potential_mobility_station.head()

In [ ]:
MIV_DISTANZ_BIS_ZIEL_KM = "MIV_Distanz_bis_Ziel_km"
MIV_ZEIT_BIS_ZIEL_MIN = "MIV_Zeit_bis_Ziel_min"

In [ ]:
pd_distances = pd.DataFrame(list(road_distances_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id.items()), columns=["ID", "MIV_Distanz_bis_Ziel_km"])
pd_distances[MIV_DISTANZ_BIS_ZIEL_KM] = pd_distances[MIV_DISTANZ_BIS_ZIEL_KM] / 1000.0
pd_durations= pd.DataFrame(list(road_durations_from_npvm_zones_with_potential_mobility_station_to_dest_per_npvm_zone_id.items()), columns=["ID", "MIV_Zeit_bis_Ziel_min"])
pd_durations[MIV_ZEIT_BIS_ZIEL_MIN] = pd_durations[MIV_ZEIT_BIS_ZIEL_MIN] / 60.0

gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station, pd_distances, on=["ID"])
gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_durations, on=["ID"])


In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data.head()

### Get pt skims from origin to potential Mobility stations

In [ ]:
OEV_JRTA_VON_START_MIN = "OEV_JRTA_von_Start_min"
OEV_NTR_VON_START = "OEV_NTR_von_Start"

In [ ]:
zone_ids_list = [x.item() for x in gdf_npvm_zones_with_potential_mobility_station[["ID"]].values]
jrta_list = [(x, get_jrta(orig_zone_id, x)) for x in zone_ids_list]
ntr_list = [(x, get_ntr(orig_zone_id, x)) for x in zone_ids_list]

In [ ]:
pd_jrtas = pd.DataFrame(jrta_list, columns=["ID", OEV_JRTA_VON_START_MIN])
pd_ntrs = pd.DataFrame(ntr_list, columns=["ID", OEV_NTR_VON_START])

In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_jrtas, on=["ID"])
gdf_npvm_zones_with_potential_mobility_station_with_data = pd.merge(gdf_npvm_zones_with_potential_mobility_station_with_data, pd_ntrs, on=["ID"])

In [ ]:
print(len(gdf_npvm_zones_with_potential_mobility_station_with_data))

In [ ]:
KOSTEN_CHF = "Kosten_CHF"

In [ ]:
CHF_PER_KM_MOBILITY = 0.75
MIN_PER_TRANSFER = 20.0
FILTER_FACTOR = 1.05

In [ ]:
def calc_costs(vtt_chf_per_h=20):
    gdf_npvm_zones_with_potential_mobility_station_with_data[KOSTEN_CHF] = \
        CHF_PER_KM_MOBILITY * gdf_npvm_zones_with_potential_mobility_station_with_data[MIV_DISTANZ_BIS_ZIEL_KM] + \
        (gdf_npvm_zones_with_potential_mobility_station_with_data[MIV_ZEIT_BIS_ZIEL_MIN] + gdf_npvm_zones_with_potential_mobility_station_with_data[OEV_JRTA_VON_START_MIN] + 
        gdf_npvm_zones_with_potential_mobility_station_with_data[OEV_NTR_VON_START] * MIN_PER_TRANSFER) / 60.0 * vtt_chf_per_h
    return gdf_npvm_zones_with_potential_mobility_station_with_data.sort_values(by=KOSTEN_CHF, ascending=True)

In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data = calc_costs(20)

In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Samedan"]

In [ ]:
def get_best_mobility_stations_per_vtt(vtt_chf_per_h):
    df_tmp = calc_costs(vtt_chf_per_h=vtt_chf_per_h)
    min_cost = df_tmp[KOSTEN_CHF].min()
    df_tmp = df_tmp[df_tmp[KOSTEN_CHF] <= min_cost * FILTER_FACTOR]
    return pd.merge(gdf_mobilty_stations_with_npvm_zone, df_tmp[["ID", KOSTEN_CHF]], on="ID").sort_values(by=KOSTEN_CHF)

In [ ]:
get_best_mobility_stations_per_vtt(0)

In [ ]:
best_mobility_stations_per_vtt = {vtt: get_best_mobility_stations_per_vtt(vtt) for vtt in range(0, 101)}

In [ ]:
best_mobility_stations_per_vtt[57]

In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Steffisburg"]

In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Thun"]

In [ ]:
gdf_npvm_zones_with_potential_mobility_station_with_data[gdf_npvm_zones_with_potential_mobility_station_with_data.N_Gem == "Bern"].head(50)

### Visualize situation on map

In [ ]:
bounds = gdf_npvm_zones_with_potential_mobility_station_with_data.total_bounds
bounds = [[bounds[1], bounds[0]], [bounds[3], bounds[2]]]
bounds

In [ ]:
best_mobility_stations_per_vtt[11]

In [ ]:
def show_best_mobility_stations_per_vtt(vtt):
    geo_data = GeoData(geo_dataframe=best_mobility_stations_per_vtt[vtt],
        style={'color': 'black', 'radius':8, 'fillColor': '#3366cc', 'opacity':0.5, 'weight':1.9, 'dashArray':'2', 'fillOpacity':0.6},
        hover_style={'fillColor': 'red' , 'fillOpacity': 0.2},
        point_style={'radius': 5, 'color': 'red', 'fillOpacity': 0.8, 'fillColor': 'blue', 'weight': 3},
        name = 'Mobility-Stationen')
    for l in m.layers:
        if type(l) == GeoData:
            m.remove_layer(l)
    m.add_layer(geo_data)

In [ ]:
def on_slider_changed(event):
    show_best_mobility_stations_per_vtt(event["new"])

In [ ]:
from ipyleaflet import Map, GeoData, basemaps, LayersControl, FullScreenControl, Marker, WidgetControl
from ipywidgets import IntSlider
import geopandas
import json

vtt_slider = IntSlider(description='Zeitkosten', min=0, max=100, value=20)
vtt_slider.observe(on_slider_changed, names='value')

widget_control = WidgetControl(widget=vtt_slider, position='topright')

m = Map(center=(52.3,8.0), zoom = 3, basemap= basemaps.Esri.WorldTopoMap, scroll_wheel_zoom=True)
m.add_control(widget_control)

m.fit_bounds(bounds)
show_best_mobility_stations_per_vtt(vtt_slider.value)
m.add_control(FullScreenControl())

m.layout.width = '100%'
m.layout.height = '500px'

orig_marker = Marker(location=(orig_lon_lat[1], orig_lon_lat[0]) , draggable=False)
dest_marker = Marker(location=(dest_lon_lat[1], dest_lon_lat[0]) , draggable=False)

m.add_layer(orig_marker)
m.add_layer(dest_marker)

m